# Ingest drivers data

In [0]:
%run ../SetUp

In [0]:
json_file_path = session_helper.get_storage_account_url(folder="raw", file="drivers.json")

drivers_df = (
    spark.read
    .format("json")
    .option("inferSchema", True)
    .load(json_file_path)
)

drivers_df.display()

In [0]:
import pyspark.sql.functions as F

transformed_drivers_df = (
    drivers_df
    .withColumnRenamed("driverId", "driver_id")
    .withColumnRenamed("driverRef", "driver_ref")
    .withColumn("ingestion_date", F.current_timestamp())
    .withColumn("full_name", F.concat(F.col("name.forename"), F.lit(" "), F.col("name.surname")))
    .drop("url", "name")
    .withColumnRenamed("full_name", "name")
)

transformed_drivers_df.display()

In [0]:
spark.sql("DROP TABLE IF EXISTS dev.drivers_bronze")
transformed_drivers_df.write.format("delta").saveAsTable("dev.drivers_bronze", mode="overwrite")